## ColumnTransform
    - ColumnTransform give us fecilties to perform preprocessing together

In [56]:
import numpy as np
import pandas as pd

In [57]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

In [58]:
df = pd.read_csv('covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [59]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

In [60]:
x_train

,age,gender,fever,cough,city
99,10,Female,98.0,Strong,Kolkata
3,31,Female,98.0,Mild,Kolkata
98,5,Female,98.0,Strong,Mumbai
48,66,Male,99.0,Strong,Bangalore
53,83,Male,98.0,Mild,Delhi
...,...,...,...,...,...
93,27,Male,100.0,Mild,Kolkata
90,59,Female,99.0,Strong,Delhi
13,64,Male,102.0,Mild,Bangalore
20,12,Male,98.0,Strong,Bangalore


## WithOut Transform

In [61]:
# Adding Simple Imputer to fever col

si = SimpleImputer()
x_train_fever = si.fit_transform(x_train[['fever']])
x_test_fever = si.transform(x_test[['fever']])

# Applying One Hot Encoder to gender col
ohe = OneHotEncoder(drop='first',sparse_output=False) # sparse_output parameter is use to return array instead of sparse matrix
x_train_gender_city = ohe.fit_transform(x_train[['gender','city']])
x_test_gender_city = ohe.transform(x_test[['gender','city']])

# Applying OridinalEncoding

oe = OrdinalEncoder(categories=[['Mild','Strong']])
x_train_cough = oe.fit_transform(x_train[['cough']])
x_test_cough = oe.transform(x_test[['cough']])

# Applying Label encoding on has covid

le = LabelEncoder()
le.fit(y_train)
y_train_has_covid = le.transform(y_train)
y_test_has_covid = le.transform(y_test)

In [62]:
x_train_gender_city.shape

(80, 4)

In [63]:
# Extracting Age From x_train dataframe
x_train_age = x_train.drop(columns=['gender','fever','cough','city']).values # .values parameter give us array instead of frame
x_test_age=x_test.drop(columns=['gender','fever','cough','city']).values

In [64]:
type(x_train_age)

numpy.ndarray

In [65]:
type(x_train_gender_city)

numpy.ndarray

In [67]:
# So After preprocessing we will make them concatenate

x_train_transformed = np.concatenate((x_train_age,x_train_fever,x_train_gender_city,x_train_cough),axis=1)
x_test_transformed = np.concatenate((x_test_age,x_test_fever,x_test_gender_city,x_test_cough),axis=1)

print(x_train_transformed.shape)

(80, 7)


# Now Using ColumnTransformer

In [69]:
from sklearn.compose import ColumnTransformer

In [74]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(drop='first',sparse_output=False),['gender','city'])
    ],remainder='passthrough') # so in transform we have transformers mean the columns you want apply transformer and remainder what do you do with remaing columns

In [75]:
transformer.fit(x_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('tnf1', SimpleImputer(), ['fever']),
                                ('tnf2',
                                 OrdinalEncoder(categories=[['Mild',
                                                             'Strong']]),
                                 ['cough']),
                                ('tnf3',
                                 OneHotEncoder(drop='first',
                                               sparse_output=False),
                                 ['gender', 'city'])])

In [84]:
x_train_transformed =transformer.transform(x_train)
x_test_transformed =transformer.transform(x_test)

x_test_transformed.shape

(20, 7)

# Now applying Logistic regression

In [91]:
from sklearn.linear_model import LogisticRegression

In [92]:
lr = LogisticRegression(max_iter=500)
lr.fit(x_train_transformed,y_train)

LogisticRegression(max_iter=500)

In [93]:
y_pred = lr.predict(x_test_transformed)

In [94]:
from sklearn.metrics  import accuracy_score

In [95]:
accuracy = accuracy_score(y_test,y_pred)
print(accuracy)

0.45
